In [1]:
data_var = '2023-12-31'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8713,2023-12-31,Espanha Acb,08:30,Real Madrid,Valencia,1.15,5.65,163.5,1.80,1.99,-12.5,2.07,1.70,d2vNZDHB,0.869565,0.176991,0.555556,0.502513,0.046556,0.2,0.0,NaN,NaN,107.676,17.399270,0.161589,127.540,49.688049,0.389588,102.188,0.000,115.43,209.95,0.0,0.0,0.0,0.0,0.935877,0.070897,0.138795,-0.64,-0.128,-1.171875,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8714,2023-12-31,Eua Nba,17:00,Washington Wizards,Atlanta Hawks,3.35,1.40,249.5,1.80,2.09,6.5,2.00,1.94,Esu4fu6s,0.298507,0.714286,0.555556,0.478469,0.012793,0.6,0.6,NaN,NaN,417.296,158.340187,0.379443,254.026,38.313037,0.150823,407.394,276.602,344.30,239.56,1.0,0.0,0.0,0.0,0.580572,0.105430,0.021536,1.84,0.368,6.385870,0.279466,0.2,-0.079466,-0.42,-0.084,-4.761905,0.410788,0.3,-0.110788
8715,2023-12-31,Eua Nba,21:00,Oklahoma City Thunder,Brooklyn Nets,1.36,3.55,234.5,1.87,2.06,-8.5,2.10,1.84,A9YChJzf,0.735294,0.281690,0.534759,0.485437,0.016984,0.8,0.4,NaN,NaN,196.830,29.991209,0.152371,214.390,75.043727,0.350034,196.830,220.092,224.46,149.76,0.0,0.0,0.0,0.0,0.630780,0.068372,0.093324,1.17,0.234,1.538462,0.670370,0.8,0.129630,-0.91,-0.182,-14.010989,0.450643,0.4,-0.050643
8716,2023-12-31,Eua Nba,21:00,San Antonio Spurs,Boston Celtics,5.25,1.11,238.5,1.83,1.83,11.5,1.83,1.83,4WXGiwj0,0.190476,0.900901,0.546448,0.546448,0.091377,1.0,0.8,NaN,NaN,589.860,658.049836,1.115603,226.632,86.248063,0.380564,331.898,233.788,271.40,216.72,0.0,0.0,0.0,0.0,0.920573,0.000000,0.000000,-1.65,-0.330,-12.878788,0.338483,0.1,-0.238483,1.02,0.204,0.539216,0.620933,0.7,0.079067
8717,2023-12-31,Eua Nba,22:00,Memphis Grizzlies,Sacramento Kings,1.95,1.99,236.5,1.86,2.06,-1.5,2.09,1.87,rNWKjc56,0.512821,0.502513,0.537634,0.485437,0.015333,0.4,0.4,NaN,NaN,309.834,100.968852,0.325880,188.406,33.847861,0.179654,233.330,227.588,185.60,235.17,0.0,0.0,0.0,0.0,0.014357,0.072154,0.078567,-1.84,-0.368,-2.581522,0.443079,0.2,-0.243079,1.33,0.266,3.721805,0.548021,0.6,0.051979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8777,2023-12-31,Taiwan Liga T1,03:00,Taipei Taishin Mars,Taoyuan Leopards,2.93,1.37,205.5,1.85,1.85,3.5,1.95,1.72,U9a1VzOR,0.341297,0.729927,0.540541,0.540541,0.071224,0.0,0.0,NaN,NaN,206.948,37.909498,0.183184,192.076,58.130048,0.302641,0.000,0.000,235.62,171.13,0.0,0.0,0.0,0.0,0.513064,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8778,2023-12-31,Turquia Tkbl Feminina,08:00,Fenerbahce 2 F,Adana BK F,2.15,1.65,133.5,1.82,1.88,1.5,2.00,1.72,UL56KItJ,0.465116,0.606061,0.549451,0.531915,0.071177,0.0,0.0,NaN,NaN,158.562,86.681069,0.546670,NaN,NaN,NaN,0.000,0.000,90.10,214.92,0.0,0.0,0.0,0.0,0.186081,0.022933,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8779,2023-12-31,Eua Nba,21:00,New Orleans Pelicans,Los Angeles Lakers,1.46,3.04,229.5,1.86,2.07,-6.5,1.97,1.97,d6U8gaLm,0.684932,0.328947,0.537634,0.483092,0.013879,0.4,1.0,NaN,NaN,171.210,34.739249,0.202904,246.594,86.575427,0.351085,167.566,232.428,143.36,317.34,0.0,0.0,0.0,0.0,0.496546,0.075569,0.000000,-1.99,-0.398,-1.155779,0.611717,0.6,-0.011717,-1.02,-0.204,-10.000000,0.553091,0.4,-0.153091
8780,2023-12-31,Eua Ncaa,16:00,Colorado,Washington State,1.34,3.23,141.5,1.80,1.95,-9.5,2.00,1.71,S22qecR4,0.746269,0.3095

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:00,Eua Nba,Washington Wizards,Atlanta Hawks,249.5,1.80,1.0000,Over
1,21:00,Eua Nba,Oklahoma City Thunder,Brooklyn Nets,234.5,1.87,1.0000,Over
2,22:00,Eua Nba,Memphis Grizzlies,Sacramento Kings,236.5,1.86,1.0000,Over
3,22:00,Eua Nba,Phoenix Suns,Orlando Magic,227.5,1.81,0.9999,Over
4,21:00,Eua Nba,New Orleans Pelicans,Los Angeles Lakers,229.5,1.86,1.0000,Over
